In [6]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.colors import colorConverter
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d import Axes3D
import os
import sys
import cbm_pack.cbm_pack as cbm_pack

from matplotlib import rcParams


In [10]:
import matplotlib.pyplot as plt
import numpy as np
import cbm_pack.file_utils as file_utils
import cbm_pack.transform as transform
import cbm_pack.analysis as analysis
from copy import deepcopy
import os.path

TRIAL_TIME = 5000 # in ms
MS_PER_HOUR = 3_600_000
MAX_CR_AMP = 6

PRE_CS_COLLECT_TS = 400
POST_CS_COLLECT_TS = 400

PYTHON_SCRIPTS_PATH = "/home/seang/Dev/Scripts/Python"

DATA_INPUT_DIR = f"{PYTHON_SCRIPTS_PATH}/data/inputs"
FORGET_DATA_INPUTS_PATH = f"{PYTHON_SCRIPTS_PATH}/data/inputs/Forgetting"

EXTENSIONS = {
    "mf_raster" : ".mfr",
    "gr_raster" : ".grr",
    "go_raster" : ".gor",
    "pc_raster" : ".pcr",
    "bc_raster" : ".bcr",
    "sc_raster" : ".scr",
    "nc_raster" : ".ncr",
    "io_raster" : ".ior",
    "mf_psth" : ".mfp", 
    "gr_psth" : ".grp",
    "go_psth" : ".gop",
    "pc_psth" : ".pcp",
    "bc_psth" : ".bcp",
    "sc_psth" : ".scp",
    "nc_psth" : ".ncp",
    "io_psth" : ".iop",
    "pfpc_weights" : ".pfpcw",
    "mfnc_weights" : ".mfncw"
}

CELL_NUMS = {
    "mf" : 4096,
    "gr" : 2 ** 20,
    "go" : 4096,
    "pc" : 32,
    "bc" : 128,
    "sc" : 512,
    "nc" : 8,
    "io" : 4
}

ModuleNotFoundError: No module named 'cbm_pack.file_utils'

In [3]:
def compute_crs_isis(isis: np.ndarray, probe_trials_per_block: np.ndarray, num_blocks: np.ndarray, input_base: str):
    num_probe_trials = probe_trials_per_block * num_blocks
    pcs_isis = {}
    for isi in isis:
        num_collect_ts = PRE_CS_COLLECT_TS + isi + POST_CS_COLLECT_TS
  
        input_base_isi = f"{input_base}_{isi}"
        input_pc_file = f"{FORGET_DATA_INPUTS_PATH}/{input_base_isi}/{input_base_isi}{EXTENSIONS['pc_raster']}"
        pc_rasters = file_utils.np_arr_from_file(input_pc_file, np.uint8)
        pc_rasters = transform.reshape_raster( \
                pc_rasters, \
                CELL_NUMS["pc"], \
                num_probe_trials, \
                num_collect_ts)

        pc_forget_crs = analysis.pcs_to_crs(pc_rasters, PRE_CS_COLLECT_TS, POST_CS_COLLECT_TS, isi)
        pcs_isis[isi] = pc_forget_crs
    return pcs_isis  
    
    # final setup stuff

In [4]:
ISIS = [200, 500, 1000, 1500]

FULL_INPUT_DIR= f"/home/satvik/lobule_v/CbmSim/data/outputs/forgetting_bugaloo/acq_no_probe_ISI/"
FULL_BASENAME=f"acq_no_probe_ISI"

FULL_CRS = calc_cr_amp_per_isi(ISIS, input_dir=FULL_INPUT_DIR, basename=FULL_BASENAME)

(500,)
ISI 200: computed cr for final cr: 5.947457938195525
(500,)
ISI 500: computed cr for final cr: 5.878672569611904
(1000,)
ISI 1000: computed cr for final cr: 5.7889670814755725


KeyboardInterrupt: 